In [2]:
import librosa 
import torch
from torch import nn
import soundfile as sf
import numpy as np
from matplotlib import pyplot as plt

In [14]:
def load_data():
    def normalize(input,target):
        mfcc_min = torch.min(input)
        mfcc_max = torch.max(input)
        input = (input - mfcc_min) / (mfcc_max - mfcc_min)

        print(target)
        mfcc_min2 = torch.min(target)
        mfcc_max2 = torch.max(target)
        target = (target - mfcc_min2) / (mfcc_max2 - mfcc_min2)
        print(torch.min(target))
        print(torch.max(target))
        print(target)
        return input,target,mfcc_min2,mfcc_max2
    

    song,sr = librosa.load("test/012146 [music].wav")
    #song, _ = librosa.magphase(song)
    mel = librosa.feature.melspectrogram(y=song,sr=sr,n_fft=2048,hop_length=512)
    #mel= librosa.power_to_db(mel, ref=np.min)
    #librosa.display.specshow(data=mel,x_axis="time",y_axis="mel")
    #plt.colorbar(format='%+2.0f dB')
    input= torch.Tensor(mel) 

    drum,sr2=librosa.load("test/012146 [drums].wav")
    drum, _ = librosa.magphase(drum)
    label = librosa.feature.melspectrogram(y=drum,sr=sr2,n_fft=2048,hop_length=512)
    label= librosa.power_to_db(label)
    

    """ librosa.display.specshow(data=label,x_axis="time",y_axis="mel")
    plt.colorbar(format='%+2.0f dB') """
    target = torch.Tensor(label) 
    

     
    
        
    return normalize(input,target)

    

load_data()

tensor([[  7.9914,  22.2119,  27.1174,  ...,  -7.8436,  -6.8130,  -6.8313],
        [  0.9804,   5.9902,   8.9781,  ..., -20.5102, -20.9072, -18.0265],
        [ -9.0411,   3.2090,   8.1580,  ..., -18.9686, -20.5764, -19.6286],
        ...,
        [-28.4808, -15.5823, -16.1480,  ..., -42.0983, -41.5222, -45.1540],
        [-32.9772, -21.3262, -22.2353,  ..., -41.1312, -43.8570, -48.3076],
        [-34.0898, -24.6650, -26.9874,  ..., -41.7350, -43.5231, -47.1973]])
tensor(0.)
tensor(1.)
tensor([[0.7152, 0.8929, 0.9542,  ..., 0.5172, 0.5301, 0.5299],
        [0.6275, 0.6901, 0.7275,  ..., 0.3589, 0.3539, 0.3899],
        [0.5023, 0.6554, 0.7172,  ..., 0.3782, 0.3581, 0.3699],
        ...,
        [0.2593, 0.4205, 0.4134,  ..., 0.0890, 0.0962, 0.0508],
        [0.2031, 0.3487, 0.3373,  ..., 0.1011, 0.0671, 0.0114],
        [0.1891, 0.3070, 0.2779,  ..., 0.0936, 0.0712, 0.0253]])


(tensor([[3.4869e-06, 3.5090e-05, 3.7245e-05,  ..., 1.2123e-04, 1.2415e-04,
          6.8595e-05],
         [2.1949e-06, 1.4804e-05, 1.4066e-05,  ..., 8.2509e-04, 1.7922e-03,
          1.2612e-03],
         [6.3812e-07, 5.6054e-06, 2.5213e-05,  ..., 1.9554e-03, 5.9118e-03,
          5.4022e-03],
         ...,
         [2.2719e-09, 3.3459e-08, 1.0234e-07,  ..., 6.6871e-09, 1.7154e-08,
          6.7842e-08],
         [2.0751e-09, 1.4556e-08, 7.3961e-08,  ..., 1.8965e-09, 1.3494e-08,
          6.2712e-08],
         [1.0184e-10, 1.3141e-09, 9.0720e-09,  ..., 1.5110e-10, 6.1186e-09,
          3.2221e-08]]),
 tensor([[0.7152, 0.8929, 0.9542,  ..., 0.5172, 0.5301, 0.5299],
         [0.6275, 0.6901, 0.7275,  ..., 0.3589, 0.3539, 0.3899],
         [0.5023, 0.6554, 0.7172,  ..., 0.3782, 0.3581, 0.3699],
         ...,
         [0.2593, 0.4205, 0.4134,  ..., 0.0890, 0.0962, 0.0508],
         [0.2031, 0.3487, 0.3373,  ..., 0.1011, 0.0671, 0.0114],
         [0.1891, 0.3070, 0.2779,  ..., 0.0936, 0.0

In [15]:
class Network(nn.Module): #NOT BATCHED FOR NOW
    def __init__(self,latspa):
        super().__init__()
        self.latspa =latspa
        self.cnn= nn.Sequential(
            nn.Conv2d(1,4,3,1,1),
            nn.Conv2d(4,8,3,1,1),
            nn.Conv2d(8,16,3,1,1),
            nn.Conv2d(16,1,3,1,1)
            
            
        )
        self.rnn1=nn.LSTM(1293,512)
        self.rnn2=nn.LSTM(512,512)
        self.rnn3=nn.LSTM(512,256)
        self.rnn4=nn.LSTM(256,128)
        
        
        
        self.encoder=nn.Sequential(
            
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,latspa)
    
        )
        self.mu = nn.Linear(latspa,latspa)
        self.dev = nn.Linear(latspa,latspa)
        
        self.decoder=nn.Sequential(

            nn.Linear(latspa,32),
            nn.ReLU(),
            nn.Linear(32,64),
            nn.ReLU(),
            nn.Linear(64,128),
            nn.ReLU(),
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Linear(256,1024),
            nn.ReLU(),
            nn.Linear(1024,165504),
            nn.Sigmoid()
           
            
        )
        
    def encode(self,nm):
        
        nm=nm.unsqueeze(0)
        cnn = self.cnn(nm)
        cnn=cnn[0]

        out,_ = self.rnn1(cnn)
        out,_ = self.rnn2(out)
        out,_ = self.rnn3(out)
        out,_ = self.rnn4(out)
        out = out[-1]

        a = self.encoder(out)
        mu = self.mu(a)
        dev = self.dev(a)

        return mu,dev
    
    def reparametrize(self, mu, log_var):

        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self,mfcc):
        
        mu,dev=self.encode(mfcc)
        z = self.reparametrize(mu,dev)
        temp = self.decoder(z)
        return  temp.view(128,1293) ,mu,dev
    
    def sample(self):
        with torch.no_grad():
            z = torch.randn(1,self.latspa)
            return self.decoder(z)


In [ ]:
def loss_function(a,mu,det,target,beta):
    
    L1= nn.functional.mse_loss(a,target)
    L2= beta*(-0.5 * torch.sum(1 + det - mu.pow(2) - det.exp()))
    
    return L1 +L2


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 1e-4
latspa=8 
model=Network(latspa)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
epoch=1000




def train_loop(input,target):
    for _ in range(epoch):
        input = input.to(device)
        target = target.to(device)
        a,mu,det= model(input)
        optimizer.zero_grad()
        loss=loss_function(a,mu,det,target)
        loss.backward()
        
        
        
    

In [17]:

input,target,mfccmin,mfccmax= load_data()
train_loop(input,target)



tensor([[  7.9914,  22.2119,  27.1174,  ...,  -7.8436,  -6.8130,  -6.8313],
        [  0.9804,   5.9902,   8.9781,  ..., -20.5102, -20.9072, -18.0265],
        [ -9.0411,   3.2090,   8.1580,  ..., -18.9686, -20.5764, -19.6286],
        ...,
        [-28.4808, -15.5823, -16.1480,  ..., -42.0983, -41.5222, -45.1540],
        [-32.9772, -21.3262, -22.2353,  ..., -41.1312, -43.8570, -48.3076],
        [-34.0898, -24.6650, -26.9874,  ..., -41.7350, -43.5231, -47.1973]])
tensor(0.)
tensor(1.)
tensor([[0.7152, 0.8929, 0.9542,  ..., 0.5172, 0.5301, 0.5299],
        [0.6275, 0.6901, 0.7275,  ..., 0.3589, 0.3539, 0.3899],
        [0.5023, 0.6554, 0.7172,  ..., 0.3782, 0.3581, 0.3699],
        ...,
        [0.2593, 0.4205, 0.4134,  ..., 0.0890, 0.0962, 0.0508],
        [0.2031, 0.3487, 0.3373,  ..., 0.1011, 0.0671, 0.0114],
        [0.1891, 0.3070, 0.2779,  ..., 0.0936, 0.0712, 0.0253]])


In [ ]:
input,target,mfccmin,mfccmax= load_data()
audio,mu,det=model(input)


audio =audio * (mfccmin-mfccmax) +mfccmin

aud=audio.detach().cpu().numpy()
aud=librosa.db_to_power(aud)
print(aud)
aud = librosa.feature.inverse.mel_to_audio(M=aud)
print(aud)
sf.write('stereo_file.wav', data= aud, samplerate=22050,subtype='PCM_24') 

tensor([[  7.9914,  22.2119,  27.1174,  ...,  -7.8436,  -6.8130,  -6.8313],
        [  0.9804,   5.9902,   8.9781,  ..., -20.5102, -20.9072, -18.0265],
        [ -9.0411,   3.2090,   8.1580,  ..., -18.9686, -20.5764, -19.6286],
        ...,
        [-28.4808, -15.5823, -16.1480,  ..., -42.0983, -41.5222, -45.1540],
        [-32.9772, -21.3262, -22.2353,  ..., -41.1312, -43.8570, -48.3076],
        [-34.0898, -24.6650, -26.9874,  ..., -41.7350, -43.5231, -47.1973]])
tensor(0.)
tensor(1.)
tensor([[0.7152, 0.8929, 0.9542,  ..., 0.5172, 0.5301, 0.5299],
        [0.6275, 0.6901, 0.7275,  ..., 0.3589, 0.3539, 0.3899],
        [0.5023, 0.6554, 0.7172,  ..., 0.3782, 0.3581, 0.3699],
        ...,
        [0.2593, 0.4205, 0.4134,  ..., 0.0890, 0.0962, 0.0508],
        [0.2031, 0.3487, 0.3373,  ..., 0.1011, 0.0671, 0.0114],
        [0.1891, 0.3070, 0.2779,  ..., 0.0936, 0.0712, 0.0253]])
[[1.3338641e-09 1.1576993e-09 9.8089159e-10 ... 1.2593799e-09
  1.2445700e-09 1.0242175e-09]
 [1.2530161e-09 1